# Function Annotations

via [Real Python](https://realpython.com/python-type-checking/)

In [25]:
import math

In [3]:
def circumference(radius: float) -> float:
    return 2 * math.pi * radius

In [4]:
circumference(1.123)

7.056017099962675

In [5]:
circumference.__annotations__

{'radius': float, 'return': float}

In [6]:
# Use with mypy to reveal the type of a given variable
# reveal_type(math.pi)
# This will show all variables
# reveal_locals()

NameError: name 'reveal_type' is not defined

Variable Annotations

In [7]:
pi: float = 3.142

In [8]:
__annotations__

{'pi': float}

# Diving Deeper

In [10]:
import random

SUITS = "♠ ♡ ♢ ♣".split()
RANKS = "2 3 4 5 6 7 8 9 10 J Q K A".split()

def create_deck(shuffle=False):
    """Create a new deck of 52 cards"""
    deck = [(s, r) for r in RANKS for s in SUITS]
    if shuffle:
        random.shuffle(deck)
    return deck

def deal_hands(deck):
    """Deal out the cards into four hands"""
    return (deck[0::4], deck[1::4], deck[2::4], deck[3::4])

def play():
    """Play a 4-player card game"""
    deck = create_deck(shuffle=True)
    names = ["P1", "P2", "P3", "P4"]
    hands = {n: h for n, h in zip(names, deal_hands(deck))}
    
    for name, cards in hands.items():
        card_str = " ".join(f"{s}{r}" for (s, r) in cards)
        print(f"{name}: {card_str}")


In [11]:
play()

P1: ♠7 ♡4 ♢5 ♣2 ♢10 ♢4 ♣9 ♣4 ♢6 ♡J ♣10 ♠10 ♣3
P2: ♢J ♡9 ♠3 ♠5 ♢Q ♣J ♣A ♡6 ♠J ♡10 ♣7 ♠6 ♣Q
P3: ♡3 ♡Q ♢K ♢7 ♣K ♣6 ♢3 ♢2 ♡8 ♠9 ♢9 ♠A ♠K
P4: ♢8 ♡A ♡K ♡7 ♡2 ♡5 ♢A ♠4 ♣8 ♣5 ♠8 ♠2 ♠Q


But now we want to compose our composite types. Let's try using `typing`

In [12]:
from typing import Dict, List, Tuple

In [13]:
names: List[str] = ["Guido", "Jukka", "Ivan"]
version: Tuple[int, int, int] = (3, 8, 9)
options: Dict[str, bool] = {"centered": False, "caps": True}

Returning to the Card Game:

In [14]:
def create_deck(shuffle: bool = False) -> List[Tuple[str, str]]:
    """Create a new deck of 52 cards"""
    docker = [(s, r) for r in RANKS for s in SUITS]
    if shuffle:
        random.shuffle(deck)
    return deck

Supposing we just had a sequence and didn't care if it was a `list` or `tuple`:

In [15]:
from typing import List, Sequence

def square(elems: Sequence[float]) -> List[float]:
    return [x**2 for x in elems]

In [16]:
square((1,2,4,5,10))

[1, 4, 16, 25, 100]

In [18]:
square([1,2,4,5,10])

[1, 4, 16, 25, 100]

Now consider how `deal_hands` would look if we annotated all the possiblities... not great.

So instead, we need to define our own type aliases

In [19]:
from typing import List, Tuple

Card = Tuple[str, str]
Deck = List[Card]

In [21]:
def deal_hands(deck: Deck) -> Tuple[Deck, Deck, Deck, Deck]:
    """Deal the cards in the deck into 4 hands"""
    return (deck[0::4], deck[1::4], deck[2::4], deck[3::4])

In [22]:
Deck

typing.List[typing.Tuple[str, str]]

We should also denote when functions don't return a value

In [23]:
def play() -> None:
    """Play a 4-player card game"""
    deck = create_deck(shuffle=True)
    names = ["P1", "P2", "P3", "P4"]
    hands = {n: h for n, h in zip(names, deal_hands(deck))}
    
    for name, cards in hands.items():
        card_str = " ".join(f"{s}{r}" for (s, r) in cards)
        print(f"{name}: {card_str}")

Something interesting: sometimes we know a function will never return normally. So use `NoReturn`

In [24]:
from typing import NoReturn

def black_hole() -> NoReturn:
    raise Exception("THERE IS NO GOING BACK!")

### Taking it altogether

In [26]:
import random
from typing import List, Tuple

SUITS = "♠ ♡ ♢ ♣".split()
RANKS = "2 3 4 5 6 7 8 9 10 J Q K A".split()

Card = Tuple[str, str]
Deck = List[Card]

def create_deck(shuffle: bool = False) -> Deck:
    """Create a new deck of 52 cards"""
    deck = [(s, r) for r in RANKS for s in SUITS]
    if shuffle:
        random.shuffle(deck)
    return deck

def deal_hands(deck: Deck) -> Tuple[Deck, Deck, Deck, Deck]:
    """Deal out the cards into four hands"""
    return (deck[0::4], deck[1::4], deck[2::4], deck[3::4])

def play() -> None:
    """Play a 4-player card game"""
    deck = create_deck(shuffle=True)
    names = ["P1", "P2", "P3", "P4"]
    hands = {n: h for n, h in zip(names, deal_hands(deck))}
    
    for name, cards in hands.items():
        card_str = " ".join(f"{s}{r}" for (s, r) in cards)
        print(f"{name}: {card_str}")

## The `Any` Type

If you have a mix of types allowed, the `Any` type is what you'd go with

In [28]:
import random
from typing import Any, Sequence

def choose(items: Sequence[Any]) -> Any:
    return random.choice(items)

But there's a better way

## SubTypes

In [33]:
int(False)

0

In [32]:
int(True)

1

In [30]:
True + True

2

In [31]:
issubclass(bool, int)

True

In [35]:
def double(number: int) -> int:
    return number * 2
print(double(True))  # Note: This would pass mypy

2


## Preserving Type Definitions (Avoiding `Any`)

In [38]:
import random
from typing import Sequence, TypeVar

Choosable = TypeVar("Chooseable")

def choose(items: Sequence[Choosable]) -> Choosable:
    return random.choices(items)

names = ["Guido", "Jukka", "Ivan"]

## Structural Subtyping

- **nominal**: comparison between types are based on names and declarations. Python typing is mostly **nominal**, eg an `int` can be used in the place of a `float` because of a subtype relationship.
- **structural**: comparisons between types are based on structure. One could define a structural type `Sized` that includes all instances defining `.__len__()` regardless of nominal type.

### Protocols

In python, via [PEP 544](https://www.python.org/dev/peps/pep-0544/) we bring structural typing via protocols. `mypy` has [implemented most of PEP 544](https://mypy.readthedocs.io/en/latest/protocols.html).

In [39]:
from typing import Sized

# `Sized` will check that `.__len__()` has been implemented

def len(obj: Sized) -> int:
    return obj.__len__()

#### Define your own Protocols

In [41]:
from typing_extensions import Protocol
# Must be explicitly installed via PyPI: `pip install typing-extensions`

class Sized(Protocol):
    def __len__(self) -> int: ...

def len(obj: Sized) -> int:
    return obj.__len__()

### The `Optional` Type

In [42]:
def player_order(names, start=None):
    """Rotate player order so that start goes first"""
    if start is None:
        start = choose(names)
    start_idx = names.index(start)
    return names[start_idx:] + names[:start_idx]

In [ ]:
from typing import Sequence, Optional

def player_order(
    names: Sequence[str],
    start: Optional[str] = None
) -> Sequence[str]:
    start_idx = names.index(start)
    return names[start_idx:] + names[:start_idx]



Notes:

- `mypy` will assume a default argument of `None` even if you don't hint `Optional`
- You can use `--no-implicit-optional` to disable this.


```bash
$ mypy player_order.py

player_order.py:8: error: Argument 1 to "index" of "list" has incompatible
                          type "Optional[str]"; expected "str"
```

See how there's now a warning to handle the `None` case?

## Object-Oriented

In [43]:
from datetime import date
from typing import Type, TypeVar

TAnimal = TypeVar("TAnimal", bound="Animal")

class Animal:
    def __init__(self, name: str, birthday: date) -> None:
        self.name = name
        self.birthday = birthday

    @classmethod
    def newborn(cls: Type[TAnimal], name: str) -> TAnimal:
        return cls(name, date.today())

    def twin(self: TAnimal, name: str) -> TAnimal:
        cls = self.__class__
        return cls(name, self.birthday)

class Dog(Animal):
    def bark(self) -> None:
        print(f"{self.name} says woof!")

fido = Dog.newborn("Fido")
pluto = fido.twin("Pluto")
fido.bark()
pluto.bark()

Fido says woof!
Pluto says woof!


## `*args, **kwargs`

In [44]:
class Game:
    def __init__(self, *names: str) -> None:
        deck = Deck.create(shuffle=True)
        self.names = (list(names) + "P1 P2 P3 P4".split())[:4]
        self.hands = {
            n: Player(n, h) for n, h in zip(self.names, deck.deal(4))
        }

Note that in this case, we don't use `*names: Sequence[str]`, but rather **`*names: str`**

## Callables

If you need to hint a function as an argument or parameter to another function, then you'd need to use `Callable`

In [46]:
from typing import Callable

def do_twice(func: Callable[[str], str], argument: str) -> None:
    print(func(argument))
    print(func(argument))

def create_greeting(name: str) -> str:
    return f"Hello {name}"

do_twice(create_greeting, "Jekyll")

Hello Jekyll
Hello Jekyll


In [47]:
from typing import Callable
from mypy_extensions import (Arg, DefaultArg, NamedArg,
                             DefaultNamedArg, VarArg, KwArg)

def func(__a: int,  # This convention is for nameless arguments
         b: int,
         c: int = 0,
         *args: int,
         d: int,
         e: int = 0,
         **kwargs: int) -> int:
    ...

F = Callable[[int,  # Or Arg(int)
              Arg(int, 'b'),
              DefaultArg(int, 'c'),
              VarArg(int),
              NamedArg(int, 'd'),
              DefaultNamedArg(int, 'e'),
              KwArg(int)],
             int]

f: F = func

Via [mypy extended callables](https://mypy.readthedocs.io/en/latest/additional_features.html#extended-callable-types)